# Lab 5

You are tasked with evaluating card counting strategies for black jack. In order to do so, you will use object oriented programming to create a playable casino style black jack game where a computer dealer plays against $n$ computer players and possibily one human player. If you don't know the rules of blackjack or card counting, please google it. 

A few requirements:
* The game should utilize multiple 52-card decks. Typically the game is played with 6 decks.
* Players should have chips.
* Dealer's actions are predefined by rules of the game (typically hit on 16). 
* The players should be aware of all shown cards so that they can count cards.
* Each player could have a different strategy.
* The system should allow you to play large numbers of games, study the outcomes, and compare average winnings per hand rate for different strategies.

1. Begin by creating a classes to represent cards and decks. The deck should support more than one 52-card set. The deck should allow you to shuffle and draw cards. Include a "plastic" card, placed randomly in the deck. Later, when the plastic card is dealt, shuffle the cards before the next deal.

2. Now design your game on a UML diagram. You may want to create classes to represent, players, a hand, and/or the game. As you work through the lab, update your UML diagram. At the end of the lab, submit your diagram (as pdf file) along with your notebook. 

3. Begin with implementing the skeleton (ie define data members and methods/functions, but do not code the logic) of the classes in your UML diagram.

4. Complete the implementation by coding the logic of all functions. For now, just implement the dealer player and human player.

5.  Test. Demonstrate game play. For example, create a game of several dealer players and show that the game is functional through several rounds.

6. Implement a new player with the following strategy:

    * Assign each card a value: 
        * Cards 2 to 6 are +1 
        * Cards 7 to 9 are 0 
        * Cards 10 through Ace are -1
    * Compute the sum of the values for all cards seen so far.
    * Hit if sum is very negative, stay if sum is very positive. Select a threshold for hit/stay, e.g. 0 or -2.  

7. Create a test scenario where one player, using the above strategy, is playing with a dealer and 3 other players that follow the dealer's strategy. Each player starts with same number of chips. Play 50 rounds (or until the strategy player is out of money). Compute the strategy player's winnings. You may remove unnecessary printouts from your code (perhaps implement a verbose/quiet mode) to reduce the output.

8. Create a loop that runs 100 games of 50 rounds, as setup in previous question, and store the strategy player's chips at the end of the game (aka "winnings") in a list. Histogram the winnings. What is the average winnings per round? What is the standard deviation. What is the probabilty of net winning or lossing after 50 rounds?


9. Repeat previous questions scanning the value of the threshold. Try at least 5 different threshold values. Can you find an optimal value?

10. Create a new strategy based on web searches or your own ideas. Demonstrate that the new strategy will result in increased or decreased winnings. 

Rachel's notes; free to ignore

Rules from: https://bicyclecards.com/how-to-play/blackjack/

Blackjack
- as close to 21 without going over 21
- Betting
    - place bets before deal begins
    - minimum and maximum limits (GR: $2-$500)
    - will ignore splitting pairs (player will not get option to split pairs)
    - will ignore insurance and doubling down
    - if dealer goes over 21, each player that has stood is paid their bet
    - if dealer at 21 or less, pays the bet of any player having a higher total and collects from those with a lower total
        - if dealer and player have equal total, no pay/collection

Game
- to start each player + dealer is given one card faced up, then a second card face up to each player, but dealer gets their second card face down
- stand/hit
    - if bust (over 21), dealer collects the bet wagered
    - Ace can either count as 1 or 11 depending on the player's hand
- Dealer's play
    - once all players have been served by the dealer, the dealer's face down card is turned up
    - Rule: hit when <=16
- drawing plastic card results in shuffling

In [1]:
import numpy as np

In [13]:
# Create some virtual classes

class base:
    def __init__(self,name):
        self.__name=name

    def name(self):
        return self.__name
    

class data(base):
    def __init__(self,name):
        base.__init__(self,name)
        
class alg(base):
    def __init__(self,name):
        base.__init__(self,name)


In [58]:
import numpy as np
import random

#Data classes

class cards(data):
    def __init__(self,suit,value):
        self.__suit=suit
        self.__value=value
    
    #accessors
    def suit(self):
        return self.__suit
    
    def value(self):
        return self.__value
    
    def show_card(self):
        print(self.__suit, self.__value)

    def __str__(self):
        return str((self.__suit,self.__value))
    
    __repr__=__str__

class game_deck(data):
    def __init__(self,n): #where n is the number of decks
        self.__n=n
        self.__deck=[] #list to represent game deck
        self.plastic=cards('plastic','plastic')

        #making game deck
        suits=['Clubs','Diamonds','Hearts','Spades']
        face_cards=['Jack','Queen','King','Ace']

        #add n number of decks to the deck list
        for l in range(self.__n):
            for i in range(4):
                #adding numbered cards
                for j in range(2,11): self.__deck.append(cards(suits[i],j)) 
                #adding face cards
                for k in range(4): self.__deck.append(cards(suits[i],face_cards[k]))

        #adding plastic card
        self.__deck.append(self.plastic)

    def __getitem__(self,key):
        return self.__deck[key]
    
    def draw(self):
        #top card is first card in list
        top_card=self.__deck[0]
        #shuffle if card is the plastic card
        if top_card==self.plastic:
            self.__deck=self.__deck.shuffledeck()
            return self.__deck.draw()
        else:
            return top_card

    #method to shuffle deck
    def shuffledeck(self):
        self.__deck=random.sample(self.__deck,len(self.__deck))
        return self.__deck


In [48]:
#testing card class
c=cards('Spades',10)
print(c.suit(),c.value())
c.show_card()

Spades 10
Spades 10


In [59]:
#testing deck class
d=game_deck(2)

#checking that first card is what is drawn
print(d[0])
print(d.draw())
print() #for visual space

#checking shuffle
shuf=d.shuffledeck()
shuf[0]

('Clubs', 2)
('Clubs', 2)



('Diamonds', 'King')

In [81]:
#Data classes

class Player(data):
    def __init__(self):
        self.__hand=[]
        self.__chips=1000 #arbitrary amount that I decided players will start with
        self.__total=0
        self.__bet=0

    def player_hand(self):
        return self.__hand
    
    def hand_total(self):
        for card in self.__hand:
            if isinstance(card.__value,int):
                self.__total+=card.__value
            elif isinstance(card.__value,str):
                if card.__value!='Ace':
                    self.__total+=10
                else:
                    if self.__total<11:
                        self.__total+=11
                    else:
                        self.__total+=1
        return self.__total
    
    def player_chips(self):
        return self.__chips
    
    def player_bet(self):
        return self.__bet
    
class Dealer(Player):
    def __init__(self):
        Player.__init__(self)

class Human(Player):
    def __init__(self):
        Player.__init__(self)

class Q6Player(Player):
    def __init__(self):
        Player.__init__(self)

    def card_counting(self,knowncards):
        #initial start with 0 value
        comp=0

        #iterating through knowncards and summing values
        for card in knowncards:
            if card.__value<7 and card.__value>=2:
                comp+=1
            elif card.__value<10 and card.__value>=7:
                comp+=0
            else:
                comp+=1
        
        #computated sum returned as output to be used to hit/stand
        return comp

    
class Blackjack(data):
    def __init__(self,number_decks,rounds,players):
        self.__number_decks=number_decks
        self.__rounds=rounds
        self.__players=players #players should be a list of Player objects
        self.__dealer=Dealer()
        self.__gamedeck=game_deck(self.__number_decks)
        self.__known=[]
        self.__hidden=[]

    #accessors
    def number_decks(self):
        return self.__number_decks
    
    def players(self):
        return self.__players

    def rounds(self):
        return self.__rounds
    
    def dealer(self):
        return self.__dealer
    
    def gamedeck(self):
        return self.__gamedeck
    
    def known(self):
        return self.__known
    
    def hidden(self):
        return self.__hidden

In [61]:
#testing Player class
player1=Player()
player1.player_hand() #should return empty list


[]

In [66]:
#testing Player class continued; all should reflect initialization numbers
print(player1.hand_total()) 
print(player1.player_bet())
print(player1.player_chips())

None
0
1000


In [83]:
#testing Blackjack class
player2=Player()
b=Blackjack(2,5,[player1,player2])
print(b.gamedeck()[53]) #check that gamedeck is actually two decks combined
print()

#testing rounds accessor; expecting 5
print(b.rounds())
print()

#testing other attribute accessors, expecting list of Player class objects and empty lists
print(b.players())
print(b.known())
print(b.hidden())

('Clubs', 3)

5

[<__main__.Player object at 0x1074675c0>, <__main__.Player object at 0x1074f65d0>]
[]
[]


In [24]:
#Algorithm classes

class initial_setup(alg):
    def __init__(self,name):
        alg.__init__(self,name)
    
    def initial_hand(self,a_player,deck):
        #give a card and removes from deck
        card=deck.draw()
        a_player.__hand.append(card)
        deck.__known.append(card)
        deck.remove(card)

    #method for giving dealer their second card face down
    def initial_dealer(self,dealer,deck):
        card=deck.draw()
        dealer.__hand.append(card)
        deck.__hidden.append(card)
        deck.remove(card)

class place_bet(alg):
    def __init__(self,name):
        alg.__init__(self,name)

    def bet(self,a_player):
        if not isinstance(a_player,[Player,Dealer,Human,Q6Player]):
            return TypeError
        elif isinstance(a_player,Human):
            hbet=int(input('Bet amount: '))
            while a_player.__chips-hbet<0:
                hbet=int(input('Bet amount: '))
            return hbet
        #general rule for other players (for convenience)
        else:
            player_bet=5
            if a_player.__chips-player_bet<0:
                print(f'{a_player} is out of chips')
            else:
                return player_bet

class pay_collect(alg):
    def __init__(self,name):
        alg.__init__(self,name)

    def paycollect(self):
        for player in self.__blackjack.__players:
            #collects bet if player busts
            if player.__total>21:
                player.__chips-=player.__bet
            #player collects if (player total>dealer total) or (dealer bust and player<21)
            elif player.__total>self.__blackjack.__dealer.__total or (self.__blackjack.__dealer.__total>21 and player<=21):
                player.__chips+=player.__bet
            #note: no chips paid/collected if player total = dealer total
            #print out players' chip status
            print(f'{player} chips: {player.__chips}')
            
class move:
    def __init__(self,name):
        alg.__init__(self,name)

    def player_move(self,a_player):
        #human player (asking for inputs)
        if isinstance(a_player,Human):
            #ask to hit or stand to user
            hmove=input('Hit or stand? ')
            hmove.lower()
        
            #keep requesting input until hit or stand is given
            while hmove!='hit' or hmove!='stand':
                hmove=input('Hit or stand? ')

            if hmove=='hit':
                new_card=self.__blackjack.__gamedeck.draw()
                a_player.__hand.append(new_card)
                self.__blackjack.__known.append(new_card)
                self.__blackjack.__gamedeck.remove(new_card)
            else:
                return 'stood'

        if isinstance(a_player,Dealer):
            #dealer hits on 16
            if a_player.hand_total()<17:
                new_card=self.__blackjack.__gamedeck.draw()
                a_player.__hand.append(new_card)
                self.__blackjack.__known.append(new_card)
                self.__blackjack.__gamedeck.remove(new_card)
            else:
                return 'stood'

        #Question 6 strat
        if isinstance(a_player,Q6Player):
            #hit if computed sum is less than -3, otherwise stand
            if a_player.card_counting()<-3:
                new_card=self.__blackjack.__gamedeck.draw()
                a_player.__hand.append(new_card)
                self.__blackjack.__known.append(new_card)
                self.__blackjack.__gamedeck.remove(new_card)
            else:
                return 'stood'

In [45]:
#function that wraps all the classes to perform a game of Blackjack
def BJGame(blackjack):
    #check blackjack class
    if not isinstance(blackjack,Blackjack):
        TypeError
    else: #the game initiates
        #start at round 1
        game_round=1

        #set up initial hands for players/dealer
        for i in range(2):
            #players are dealt one cards face up before looping to do so once again
            for player in blackjack.__players:
                initial_setup().initial_hand(player,blackjack.__gamedeck)
            #dealer of game is deals first card face up like players  
            if i==0:
                initial_setup().initial_hand(blackjack.__dealer,blackjack.__gamedeck)
            #dealer is deals second card to themselves face down
            elif i==1:
                initial_setup().initial_dealer(blackjack.__dealer,blackjack.__gamedeck)

        while game_round<blackjack.__rounds:
            #Print out what round
            print(f'Round {game_round}')

            #betting for round
            for player in blackjack.__players:
                player.__bet=place_bet.bet()
                #initial status to allow for looping
                player.stand_status=False
            
            #Round starts (actual card game)
            while any(player.stand_status==False):
                for player in blackjack.__player:
                    #if player calls stand, dealer stops dealing to them
                    if player.stand_status==True:
                        continue
                    #player gets dealt until calls stand
                    else:
                        mv=move(blackjack).player_move(player)
                        if mv=='stood':
                            player.stand_status=True

            #Round finished; bets paid/collect
            pay_collect().paycollect()        

            #round count updated
            game_round+=1

        print('Blackjack game ended')



In [30]:
#testing with one human player and two dealer players
p1=Human()
p2=Dealer()
p3=Dealer()
players_list=[p1,p2,p3]

bj=Blackjack(6,5,players_list)
BJGame(bj)

AttributeError: 'Blackjack' object has no attribute '__players'